In [0]:
!pwd
!wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fishRecognition_GT.tar
!tar -xf fishRecognition_GT.tar
!ls

/content
--2018-12-04 15:24:53--  http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fishRecognition_GT.tar
Resolving groups.inf.ed.ac.uk (groups.inf.ed.ac.uk)... 129.215.202.26
Connecting to groups.inf.ed.ac.uk (groups.inf.ed.ac.uk)|129.215.202.26|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 510912000 (487M) [application/x-tar]
Saving to: ‘fishRecognition_GT.tar’

fishRecognition_GT. 100%[===================>] 487.24M  24.3MB/s    in 34s     

2018-12-04 15:25:28 (14.2 MB/s) - ‘fishRecognition_GT.tar’ saved [510912000/510912000]

tar: A lone zero block at 997875
fish_image  fishRecognition_GT.tar  mask_image	sample_data


In [0]:
import os
os.chdir("/content/")
!pwd
!ls


/content
fish_image  sample_data
sample_data


In [0]:
!ls

sample_data  wget-log  wget-log.1  wget-log.2  wget-log.3


In [0]:
#谷歌云盘挂载
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive
# 挂载Google drive的位置是 /content/drive/
# import os

# path = "/content/drive/"
# os.chdir(path)
# os.listdir(path)

In [0]:
#tensorflowjs转换
!pip install tensorflowjs
!mkdir model
!tensorflowjs_converter --input_format keras model-ep030-train_loss0.000-val_loss0.019-val_acc0.996.h5 /content/drive/model #对模型进行转换
!zip -r model.zip /content/drive/model #进行压缩


In [0]:
from google.colab import files
files.download('/content/drive/model.zip') #下载

In [0]:
#模型保存和下载
model.save('/content/drive/test.h5')
from google.colab import files
files.download("/content/drive/test.h5")


In [0]:
# 分类报告：precision/recall/fi-score/均值/分类个数
path = "/content/drive/"
os.chdir(path)
os.listdir(path)
import keras
from keras.models import load_model
from keras.utils.generic_utils import CustomObjectScope
from sklearn.metrics import classification_report
with CustomObjectScope({'relu6': keras.applications.mobilenet.relu6,'DepthwiseConv2D': keras.applications.mobilenet.DepthwiseConv2D}):
    model=load_model('model-ep030-train_loss0.000-val_loss0.019-val_acc0.996.h5')
# pred_y = loadmodel.predict(x_test[1])
# print(pred_y)
# pred_label = np.argmax(pred_y, axis=1)
# true_label = np.argmax(y_test, axis=1)
# classification_report(true_label,pred_label)

fishtype=["Dascyllus reticulatus","Plectroglyphidodon dickii","Chromis chrysura","Amphiprion clarkii","Chaetodon lunulatus","Chaetodon trifascialis","Myripristis kuntee","Acanthurus nigrofuscus","Hemigymnus fasciatus","Neoniphon sammara","Abudefduf vaigiensis","Canthigaster valentini","Pomacentrus moluccensis","Zebrasoma scopas","Hemigymnus melapterus","Lutjanus fulvus","Scolopsis bilineata","Scaridae","Pempheris vanicolensis","Zanclus cornutus","Neoglyphidodon nigroris","Balistapus undulatus","Siganus fuscescens"]
#预测类别

from PIL import Image
import numpy as np



import time
t0 = time.time()
img_height=224
img_weight=224
test=np.array(Image.open('/content/drive/fish.jpg').resize((img_height, img_weight),Image.ANTIALIAS))
test=np.array(test)
test=test.reshape(-1,img_height,img_weight,3).astype("float32")/255.0
preds = model.predict(test)
y=np.argmax(model.predict(test))
print(" 预测分类为%s 概率为(%.2f%%)"  % (fishtype[y],preds[0][y]*100))
print("耗时: %.2f seconds..." % (time.time()-t0))


# t0 = time.time()
# import numpy as np
# from keras.preprocessing import image
# from keras.applications.mobilenet import preprocess_input
# from keras.applications.imagenet_utils import decode_predictions
# img = image.load_img('/content/drive/fish.jpg',target_size=(img_height,img_weight))
# x = image.img_to_array(img)
# x = np.expand_dims(x,axis=0)
# x = preprocess_input(x)
# preds = model.predict(x)
# y=np.argmax(preds)
# print(" 预测分类为%s 概率为(%.2f%%)"  % (y+1,preds[0][y]*100))
# print("耗时: %.2f seconds..." % (time.time()-t0))

Using TensorFlow backend.


 预测分类为Chaetodon trifascialis 概率为(96.56%)
耗时: 2.06 seconds...


In [0]:
path= '/content/fish_image/'
#通用
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import os
from keras import backend as K
from keras.models import Model
from PIL import Image
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import plot_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import TensorBoard
import random as rn
#可复现代码
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(2)
rn.seed(1)
tf.set_random_seed(1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

# 限制内存占用
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.95
set_session(tf.Session(config=config))

img_height=128
img_weight=128
chnnel_number=3
#数据准备
def read_image(img_name):
    im = Image.open(img_name).resize((img_height, img_weight),Image.ANTIALIAS)
    data = np.array(im)
    return data
def read_label(dirname):
    data=os.path.basename(dirname)
    return data
images = []
labels=[]
for (dirname,dirs,files) in os.walk(path):
    for file in files:
        if file.endswith('.png'):
            fd = os.path.join(dirname,file)
            images.append(read_image(fd))
            labels.append(read_label(dirname))
            
#数据分割
x = np.array(images)
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(labels)
y = np.array(encoded_Y)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size = 0.3, random_state= 0)
# img_height=28
# img_weight=28
# xx_train = x_train.reshape(y_train.shape[0],img_size).astype("float32")/255
# xx_test =  x_test.reshape(y_test.shape[0],img_size).astype("float32")/255
x_train = x_train.reshape(x_train.shape[0],img_height,img_weight,chnnel_number).astype("float16")/255.0
x_test =  x_test.reshape(x_test.shape[0],img_height,img_weight,chnnel_number).astype("float16")/255.0
# x_train = x_train.reshape(x_train.shape[0], -1) / 255.  
# x_test = x_test.reshape(x_test.shape[0], -1) / 255. 
# x_train = x_train.reshape(-1, 1,28, 28)/255.
# x_test = x_test.reshape(-1, 1,28, 28)/255.
# x_train = x_train.reshape(-1,28,28,1)/255.0
# x_test = x_test.reshape(-1,28,28,1)/255.0
y_train_value = y_train
y_test_value = y_test

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
#性能
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
#中心损失
from keras.models import Model
from keras.layers import Conv2D, Input, MaxPooling2D,Dense,Activation,Dense,GlobalAveragePooling2D,Input, Activation, add, Dense, Flatten, Dropout, Multiply, Embedding, Lambda,PReLU
# input_image =Input(shape=(img_height,img_weight,chnnel_number))
#mobilnet 未训练
mobilnet_base_model=keras.applications.mobilenet.MobileNet(input_shape=(img_height,img_weight,chnnel_number),include_top=False, weights=None)
#mobilnet 预训练
mobilnetpr_base_model=keras.applications.mobilenet.MobileNet(input_shape=(img_height,img_weight,chnnel_number),include_top=False, weights='imagenet')


nb_classes=23
lambda_c = 0.05
feature_size=1024
# 预训练
xp=mobilnetpr_base_model.output
xp=GlobalAveragePooling2D()(xp)
featurep = Dense(feature_size, activation='relu')(xp)
predictp = Dense(nb_classes, activation='softmax')(featurep) #至此，得到一个常规的softmax分类模型
#中心损失
input_target = Input(shape=(1,))
centers = Embedding(nb_classes, feature_size)(input_target) #Embedding层用来存放中心
l2_loss = Lambda(lambda x: K.sum(K.square(x[0]-x[1][:,0]), 1, keepdims=True), name='l2_loss')([feature,centers])
l2p_loss = Lambda(lambda x: K.sum(K.square(x[0]-x[1][:,0]), 1, keepdims=True), name='l2p_loss')([featurep,centers])
model_centerloss = Model(inputs=[mobilnet_base_model.input,input_target],outputs=[predict,l2_loss])        
model_centerloss.compile(optimizer=keras.optimizers.Adam(), loss=["categorical_crossentropy", lambda y_true,y_pred: y_pred],loss_weights=[1,lambda_c],metrics=[precision,recall,f1])
modelpr_centerloss = Model(inputs=[mobilnetpr_base_model.input,input_target],outputs=[predictp,l2p_loss])        
modelpr_centerloss.compile(optimizer=keras.optimizers.Adam(), loss=["categorical_crossentropy", lambda y_true,y_pred: y_pred],loss_weights=[1,lambda_c],metrics=[precision,recall,f1])



filepath = '/content/drive/model-ep{epoch:03d}-train_loss{loss:.3f}-val_loss{val_loss:.3f}-val_acc{val_acc:.3f}.h5'
checkpoint = ModelCheckpoint(filepath,monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=10)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto') 
reduce=ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
tb=TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
time=keras.callbacks.TerminateOnNaN()#当遇到 NaN 损失会停止训练的回调函数。
sample=keras.callbacks.ProgbarLogger(count_mode='samples')#会把评估以标准输出打印的回调函数
base=keras.callbacks.BaseLogger()#会积累训练轮平均评估的回调函数。
cbks=[]
cbks.append(early_stopping)
cbks.append(reduce)
# cbks.append(tb)
# cbks.append(checkpoint)
# cbks.append(time)
# cbks.append(sample)
# cbks.append(base)

epochs=100
b_size=32
v_split=0.2

random_y_train = np.random.rand(x_train.shape[0],1)
random_y_test = np.random.rand(x_test.shape[0],1)

model_centerloss.fit([x_train,y_train_value], [y_train, random_y_train], batch_size=b_size, epochs=epochs, verbose=1, validation_data=([x_test,y_test_value], [y_test,random_y_test]), callbacks=cbks)
modelpr_centerloss.fit([x_train,y_train_value], [y_train, random_y_train], batch_size=b_size, epochs=epochs, verbose=1, validation_data=([x_test,y_test_value], [y_test,random_y_test]), callbacks=cbks)


Train on 19159 samples, validate on 8211 samples
Epoch 1/100
19159/19159 [==============================] - 91s 5ms/step - loss: 1.3722 - dense_15_loss: 1.1106 - l2_loss_loss: 5.2330 - dense_15_precision: 0.8329 - dense_15_recall: 0.5086 - dense_15_f1: 0.5974 - l2_loss_precision: 0.8162 - l2_loss_recall: 1.6363 - l2_loss_f1: 1.0850 - val_loss: 1.8929 - val_dense_15_loss: 1.6776 - val_l2_loss_loss: 4.3049 - val_dense_15_precision: 0.7228 - val_dense_15_recall: 0.2743 - val_dense_15_f1: 0.3955 - val_l2_loss_precision: 0.8508 - val_l2_loss_recall: 1.7167 - val_l2_loss_f1: 1.1325
Epoch 2/100
19159/19159 [==============================] - 84s 4ms/step - loss: 0.5377 - dense_15_loss: 0.4692 - l2_loss_loss: 1.3691 - dense_15_precision: 0.9396 - dense_15_recall: 0.8510 - dense_15_f1: 0.8927 - l2_loss_precision: 0.5434 - l2_loss_recall: 0.8266 - l2_loss_f1: 0.6440 - val_loss: 0.6262 - val_dense_15_loss: 0.5117 - val_l2_loss_loss: 2.2901 - val_dense_15_precision: 0.9218 - val_dense_15_recall: 0.

In [0]:
print(x_train.shape)
print(y_test.shape)
print(x_test.shape)
print(y_train.shape)
from keras.models import Model
from keras.layers import Conv2D, Input, MaxPooling2D,Dense,Activation,Dense,GlobalAveragePooling2D
input_image =Input(shape=(img_height,img_weight,chnnel_number))
#mobilnet 未训练
mobilnet_base_model=keras.applications.mobilenet.MobileNet(input_shape=(img_height,img_weight,chnnel_number),include_top=False, weights = None)
x=mobilnet_base_model.output
x=GlobalAveragePooling2D()(x)
des1=keras.layers.Dense(1024,activation='relu')(x)
dp1=keras.layers.Dropout(0.5)(des1)
pre=keras.layers.Dense(23,activation=tf.nn.softmax)(dp1)
#mobilnet 预训练
mobilnetpr_base_model=keras.applications.mobilenet.MobileNet(input_shape=(img_height,img_weight,chnnel_number),include_top=False, weights='imagenet')
px=mobilnetpr_base_model.output
px=GlobalAveragePooling2D()(px)
pdes1=keras.layers.Dense(1024,activation='relu')(px)
pdp1=keras.layers.Dropout(0.5)(pdes1)
ppre=keras.layers.Dense(23,activation=tf.nn.softmax)(pdp1)
mobilnetpr = keras.models.Model(inputs=mobilnetpr_base_model.input, outputs=ppre)
mobilnet = keras.models.Model(inputs=mobilnet_base_model.input, outputs=pre)
mobilnetpr.compile(optimizer=keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=[precision,recall,f1])
mobilnet.compile(optimizer=keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=[precision,recall,f1])

filepath = '/content/drive/model-ep{epoch:03d}-train_loss{loss:.3f}-val_loss{val_loss:.3f}-val_acc{val_acc:.3f}.h5'
checkpoint = ModelCheckpoint(filepath,monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=10)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto') 
reduce=ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=3, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
tb=TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
time=keras.callbacks.TerminateOnNaN()#当遇到 NaN 损失会停止训练的回调函数。
sample=keras.callbacks.ProgbarLogger(count_mode='samples')#会把评估以标准输出打印的回调函数
base=keras.callbacks.BaseLogger()#会积累训练轮平均评估的回调函数。
cbks=[]
cbks.append(early_stopping)
cbks.append(reduce)
# cbks.append(tb)
# cbks.append(checkpoint)
# cbks.append(time)
# cbks.append(sample)
# cbks.append(base)

epochs=100
b_size=64
v_split=0.2

# history2=mobilnetpr.fit(x_train, y_train, batch_size=b_size, epochs=epochs, verbose=1, validation_split=0.25, callbacks=cbks)
# history1=mobilnet.fit(x_train, y_train, batch_size=b_size, epochs=epochs, verbose=1, validation_data=(x_test,y_test), callbacks=cbks)

from sklearn.metrics import classification_report
pred_y = mobilnetpr.predict(x_test)
pred_label = np.argmax(pred_y, axis=1)
true_label = np.argmax(y_test, axis=1)
classification_report(true_label,pred_label)






(19159, 128, 128, 3)
(8211, 23)
(8211, 128, 128, 3)
(19159, 23)
17227776/17225924 [==============================] - 0s 0us/step


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


'             precision    recall  f1-score   support\n\n          0       0.72      0.02      0.03      3664\n          1       0.15      0.09      0.11       769\n          2       0.21      0.14      0.17      1096\n          3       0.05      0.01      0.02      1201\n          4       0.00      0.00      0.00       774\n          5       0.03      0.07      0.04        58\n          6       0.00      0.00      0.00       152\n          7       0.01      0.08      0.01        59\n          8       0.00      0.00      0.00        69\n          9       0.01      0.07      0.02        83\n         10       0.00      0.00      0.00        26\n         11       0.00      0.07      0.00        43\n         12       0.08      0.07      0.07        46\n         13       0.00      0.00      0.00        31\n         14       0.01      0.17      0.02        12\n         15       0.00      0.00      0.00        58\n         16       0.00      0.00      0.00        17\n         17       0.00   

In [0]:
#alexmodel
alexmodel = keras.Sequential([    
    keras.layers.Convolution2D(96,11,strides=(4,4),input_shape=(img_height,img_weight,chnnel_number),padding='valid',activation='relu',kernel_initializer='uniform'),
    keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same'),
    keras.layers.Convolution2D(256,(5,5),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'),
    keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same'),
    keras.layers.Convolution2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'),
    keras.layers.Convolution2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'),
    keras.layers.Convolution2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'),
    keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same'),
    keras.layers.Flatten(),
    keras.layers.Dense(4096,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(23,activation=tf.nn.softmax)
])
alexmodel.compile(optimizer=keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=[precision,recall,f1])
alexmodel.fit(x_train,y_train, batch_size=b_size, epochs=epochs, verbose=1, validation_split=0.25,callbacks=cbks)
from sklearn.metrics import classification_report
pred_y = alexmodel.predict(x_test)
pred_label = np.argmax(pred_y, axis=1)
true_label = np.argmax(y_test, axis=1)
classification_report(true_label,pred_label)


NameError: ignored

In [0]:
#回调函数
filepath = '/content/drive/model-ep{epoch:03d}-train_loss{loss:.3f}-val_loss{val_loss:.3f}-val_acc{val_acc:.3f}.h5'
checkpoint = ModelCheckpoint(filepath,monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=10)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto') 
reduce=ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=3, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
tb=TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
time=keras.callbacks.TerminateOnNaN()#当遇到 NaN 损失会停止训练的回调函数。
sample=keras.callbacks.ProgbarLogger(count_mode='samples')#会把评估以标准输出打印的回调函数
base=keras.callbacks.BaseLogger()#会积累训练轮平均评估的回调函数。
cbks=[]
cbks.append(early_stopping)
cbks.append(reduce)
# cbks.append(tb)
cbks.append(checkpoint)
# cbks.append(time)
# cbks.append(sample)
# cbks.append(base)

In [0]:
#I3SVM 139*139
b_size=64
epochs=100
from keras.models import Model
from keras.layers import Conv2D, Input, MaxPooling2D,Dense,Activation,Dense,GlobalAveragePooling2D
InceptionV3_base_model=keras.applications.inception_v3.InceptionV3(input_shape=(img_height,img_weight,chnnel_number),include_top=False, weights='imagenet')
x=InceptionV3_base_model.output
x=keras.layers.Flatten()(x)
pre=keras.layers.Dense(23,activation='linear',kernel_regularizer=keras.regularizers.l2(0.01))(x)
I3SVM= keras.models.Model(inputs=InceptionV3_base_model.input, outputs=pre)
I3SVM.compile(loss='hinge',optimizer='adadelta',metrics=[precision,recall,f1])
I3SVM.fit(x_train,y_train, batch_size=b_size, epochs=epochs, verbose=1, validation_split=0.25,callbacks=cbks)
from sklearn.metrics import classification_report
pred_y = I3SVM.predict(x_test)
pred_label = np.argmax(pred_y, axis=1)
true_label = np.argmax(y_test, axis=1)
classification_report(true_label,pred_label)

Train on 16422 samples, validate on 5474 samples
Epoch 1/100
16422/16422 [==============================] - 211s 13ms/step - loss: 1.0058 - acc: 0.3383 - val_loss: 0.9570 - val_acc: 0.1394
Epoch 2/100
16422/16422 [==============================] - 191s 12ms/step - loss: 0.9570 - acc: 0.1530 - val_loss: 0.9570 - val_acc: 0.0066
Epoch 3/100
16422/16422 [==============================] - 191s 12ms/step - loss: 0.9569 - acc: 0.1246 - val_loss: 0.9569 - val_acc: 0.0431

Epoch 00003: val_loss did not improve from 0.95670
Epoch 4/100
16422/16422 [==============================] - 191s 12ms/step - loss: 0.9569 - acc: 0.1054 - val_loss: 0.9568 - val_acc: 0.0899
Epoch 5/100
16422/16422 [==============================] - 191s 12ms/step - loss: 0.9567 - acc: 0.1775 - val_loss: 0.9567 - val_acc: 0.1052
Epoch 6/100
16422/16422 [==============================] - 191s 12ms/step - loss: 0.9567 - acc: 0.2896 - val_loss: 0.9567 - val_acc: 0.3922
Epoch 7/100
16422/16422 [==============================] - 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


'             precision    recall  f1-score   support\n\n          0       0.44      1.00      0.61      2381\n          1       0.00      0.00      0.00       551\n          2       0.00      0.00      0.00       745\n          3       1.00      0.05      0.09       804\n          4       1.00      0.00      0.00       511\n          5       0.00      0.00      0.00        36\n          6       0.00      0.00      0.00        86\n          7       0.00      0.00      0.00        41\n          8       0.00      0.00      0.00        60\n          9       0.00      0.00      0.00        58\n         10       0.00      0.00      0.00        24\n         11       0.00      0.00      0.00        39\n         12       0.00      0.00      0.00        34\n         13       0.00      0.00      0.00        14\n         14       0.00      0.00      0.00        10\n         15       0.00      0.00      0.00        39\n         16       0.00      0.00      0.00         6\n         17       0.00   

In [0]:
#vgg16 and 19
import keras
import tensorflow as tf
import numpy as np 
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.layers import Conv2D, Input, MaxPooling2D,Dense,Activation,Dense,GlobalAveragePooling2D
epochs=100
b_size=128
VGG16_base_model=VGG16(input_shape=(img_height,img_weight,chnnel_number),include_top=False, weights=None)
VGG19_base_model=VGG19(input_shape=(img_height,img_weight,chnnel_number),include_top=False, weights=None)
x=VGG16_base_model.output
fx=keras.layers.Flatten()(x)
dx=keras.layers.Dense(4096,activation='relu')(fx)
dr1x=keras.layers.Dropout(0.5)(dx)
d2x=keras.layers.Dense(4096,activation='relu')(dr1x)
dr2x=keras.layers.Dropout(0.5)(d2x)
pre16=keras.layers.Dense(23,activation=tf.nn.softmax)(dr2x) 
y=VGG19_base_model.output
fy=keras.layers.Flatten()(y)
dy=keras.layers.Dense(4096,activation='relu')(fy)
dr1y=keras.layers.Dropout(0.5)(dy)
d2y=keras.layers.Dense(4096,activation='relu')(y)
dr2y=keras.layers.Dropout(0.5)(d2y)
pre19=keras.layers.Dense(23,activation=tf.nn.softmax)(dr2y) 
vgg16model= keras.models.Model(inputs=VGG16_base_model.input, outputs=pre16)
vgg19model= keras.models.Model(inputs=VGG19_base_model.input, outputs=pre19)
#vggnet19
model3 = keras.Sequential()  
model3.add(keras.layers.Convolution2D(64,(3,3),strides=(1,1),input_shape=(img_height,img_weight,chnnel_number),padding='same',activation='relu',kernel_initializer='uniform'))  
model3.add(keras.layers.Convolution2D(64,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model3.add(keras.layers.MaxPooling2D(pool_size=(2,2),padding='same'))  
model3.add(keras.layers.Convolution2D(128,(3,2),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model3.add(keras.layers.Convolution2D(128,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model3.add(keras.layers.MaxPooling2D(pool_size=(2,2),padding='same'))  
model3.add(keras.layers.Convolution2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model3.add(keras.layers.Convolution2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model3.add(keras.layers.MaxPooling2D(pool_size=(2,2),padding='same'))  
model3.add(keras.layers.Convolution2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model3.add(keras.layers.Convolution2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model3.add(keras.layers.MaxPooling2D(pool_size=(2,2),padding='same'))  
model3.add(keras.layers.Convolution2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model3.add(keras.layers.Convolution2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
model3.add(keras.layers.MaxPooling2D(pool_size=(2,2),padding='same'))  
model3.add(keras.layers.Flatten())  
model3.add(keras.layers.Dense(4096,activation='relu'))  
model3.add(keras.layers.Dropout(0.5))  
model3.add(keras.layers.Dense(4096,activation='relu'))  
model3.add(keras.layers.Dropout(0.5))  
model3.add(keras.layers.Dense(23,activation=tf.nn.softmax))  
model3.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(lr=0.0001),metrics=[precision,recall,f1]) 
vgg16model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(),metrics=[precision,recall,f1])
vgg19model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(),metrics=[precision,recall,f1])
vgg16model.fit(x_train,y_train, batch_size=b_size, epochs=epochs, verbose=1, validation_split=0.25,callbacks=cbks)
vgg19model.fit(x_train,y_train, batch_size=b_size, epochs=epochs, verbose=1, validation_split=0.25,callbacks=cbks)

from sklearn.metrics import classification_report
pred_y = vgg16model.predict(x_test)
pred_label = np.argmax(pred_y, axis=1)
true_label = np.argmax(y_test, axis=1)
classification_report(true_label,pred_label)

from sklearn.metrics import classification_report
pred_y = vgg19model.predict(x_test)
pred_label = np.argmax(pred_y, axis=1)
true_label = np.argmax(y_test, axis=1)
classification_report(true_label,pred_label)

Train on 16422 samples, validate on 5474 samples
Epoch 1/100
16422/16422 [==============================] - 215s 13ms/step - loss: 2.0027 - acc: 0.3825 - val_loss: 1.8205 - val_acc: 0.4361
Epoch 2/100
16422/16422 [==============================] - 207s 13ms/step - loss: 1.7945 - acc: 0.4472 - val_loss: 1.8188 - val_acc: 0.4361
Epoch 3/100
16422/16422 [==============================] - 207s 13ms/step - loss: 1.7938 - acc: 0.4472 - val_loss: 1.8299 - val_acc: 0.4361
Epoch 4/100
14336/16422 [=========================>....] - ETA: 23s - loss: 1.7909 - acc: 0.4480

KeyboardInterrupt: ignored

In [0]:
#损失图
plt.figure(facecolor='none')
plt.grid(False)
plt.plot(history1.history['acc'],'bs-')
plt.plot(history2.history['acc'],'g^-')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch x10')
frame=plt.legend(['MN', 'MNT'], loc='center right')
plt.show()

plt.figure(facecolor='none')
plt.grid(False)
plt.plot(history1.history['loss'],'bs-')
plt.plot(history2.history['loss'],'g^-')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('epoch x10')
plt.legend(['MN', 'MNT'], loc='center right')
plt.show()

In [0]:
#confusion matrix
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
def plot_sonfusion_matrix(cm, classes, normalize=False, title='Confusion matrix',cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.grid(False)
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
    thresh = cm.max()/2.0
    for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j,i,cm[i,j], horizontalalignment='center',color='white' if cm[i,j]>thresh else 'black')
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predict label')
 
pred_y = mobilnetpr.predict(x_test)
pred_label = np.argmax(pred_y, axis=1)
true_label = np.argmax(y_test, axis=1)
confusion_mat = confusion_matrix(true_label, pred_label)
plot_sonfusion_matrix(confusion_mat, classes = range(1,24))

In [0]:
#AUC值
from sklearn.metrics import roc_auc_score
from keras import backend as KAUC
# AUC for a binary classifier
def auc(y_true, y_pred):
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)
    binSizes = -(pfas[1:]-pfas[:-1])
    s = ptas*binSizes
    return K.sum(s, axis=0)
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# PFA, prob false alert for binary classifier
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # N = total number of negative labels
    N = K.sum(1 - y_true)
    # FP = total number of false alerts, alerts from the negative class labels
    FP = K.sum(y_pred - y_pred * y_true)
    return FP/N
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# P_TA prob true alerts for binary classifier
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # P = total number of positive labels
    P = K.sum(y_true)
    # TP = total number of correct alerts, alerts from the positive class labels
    TP = K.sum(y_pred * y_true)
    return TP/P
 

